In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import xarray as xr

import thermoextrap.xtrapy as xtrapy
import thermoextrap.xtrapy.xpan_beta as xpan_beta

# Data Organization

All of the extrapolation and interpolation models in the thermoextrap package expect input data to be organized in a certain fashion. To help manage the data, there are data objects that help organize it. Even the inputs to these data object, however, must be organized appropriately. Here we will use data from our ideal gas test system to demonstrate this organization, as well as the various options for what types of data that may be provided as input.

In [2]:
#Import idealgas module
from thermoextrap.xtrapy import idealgas

#Define reference beta
beta_ref = 5.6

#And maximum order
order = 2

npart = 1000 #Number of particles (in single configuration)
nconfig = 100_000 #Number of configurations

#Generate all the data we could want
xdata, udata = idealgas.generate_data((nconfig, npart), beta_ref)

In [3]:
help(xtrapy.data)

Help on module thermoextrap.xtrapy.data in thermoextrap.xtrapy:

NAME
    thermoextrap.xtrapy.data - Routines to handle data objects

CLASSES
    abc.ABC(builtins.object)
        AbstractData
            DataCentralMomentsBase
                DataCentralMoments
                DataCentralMomentsVals
            DataValuesBase
                DataValues
                DataValuesCentral
    builtins.object
        DatasetSelector
    
    class AbstractData(abc.ABC)
     |  Helper class that provides a standard way to create an ABC using
     |  inheritance.
     |  
     |  Method resolution order:
     |      AbstractData
     |      abc.ABC
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __len__(self)
     |  
     |  resample(self, indices=None, nrep=None, **kwargs)
     |  
     |  ----------------------------------------------------------------------
     |  Readonly properties defined here:
     |  
     |  central
     |  
     |  order
     |

Rather than passing data directly to `__init__` methods for creating data class objects and simultaneously telling it which dimensions mean what (or expecting that specific dimensions mean a certain thing), `thermoextrap` uses xarray to label the dimensions of inputs. While this is also useful in the background, it helps to clarify what is expected of user inputs.

Currently, `xdata` is of the shape (nconfig), or the number of configurations generated with each entry being the average $x$ location for the associated configuration.

In [4]:
print(nconfig, xdata.shape)

100000 (100000,)


The dimension over which independent samples vary is the "record" dimension, with its default name in `thermoextrap` being 'rec'. So when we create an xarray `DataArray` object to house the input $x$ data, we must label that dimension 'rec'. Same goes for the input potential energy data. Note that the list provided to the argument `dims` is a list of strings naming the dimensions in the array passed to `DataArray`.

In [5]:
xdata = xr.DataArray(xdata, dims=['rec'])
udata = xr.DataArray(udata, dims=['rec'])

Now when we create a data object in `thermoextrap` to hold the data, we tell it that the "record" dimension, `rec_dim` is named 'rec', which is the default, but it could be named something different as long as you provided that name to `rec_dim`.

Note that the `xv` is the argument for the observable $x$ and `uv` is the potential energy or appropriate Hamiltonian or thermodynamic conjugate variable.

<font color="red">Difference between DataCentralMoments.from_vals and DataCentralMomentsVals.from_vals?  Seems like a resampling thing?  And can this be set up so don't need DataCentralMomentsVals?</font>


<font color="blue">
DataCentralMomentsVals (vals) comes from raw values.  
DataCentralMoments (mom) from pre computed moments.  
The main difference between these is how resampling is done.  For vals, the raw values are resampled.  For mom,
the collected moments are resampled.  
You got it down below.
</font>

In [22]:
data = xpan_beta.DataCentralMomentsVals.from_vals(order=order, 
                                                  rec_dim='rec',
                                                  xv=xdata, uv=udata, central=True)

A couple more notes are in order about the inputs to any of the `thermoextrap` data object variants. First, you only need to provide the order you expect to extrapolate to up front if you're using the `from_vals` constructor. This is because you need to specify the order of moments that will be calculated from the raw data.

The next argument to be aware of is `central`. This is True by default and tells the data object to work with central moments for calculating derivatives in the background, which it turns out is much more numerically stable than non-central moments. You probably want `central` to be True, but know that you can change it if you wish.

A lot of data is already computed as soon as we create our data object. The original raw data is still stored in `.xv` and `.uv`, and order is `.order`, but we can already see the central moments if we look at `.values`.

In [90]:
data.xave

<xarray.DataArray ()>
array(0.17486323)

In [91]:
data.u

<xarray.DataArray (umom: 3)>
array([1.00000000e+00, 1.74863233e+02, 3.06051107e+04])
Dimensions without coordinates: umom

In [94]:
data.dxdu

<xarray.DataArray (umom: 3)>
array([0.        , 0.02796055, 0.00718139])
Dimensions without coordinates: umom

Understanding this internal structure will help to understand possible inputs as well. In `.values`, the data object has stored the central moments of $\langle x \rangle$ and $\langle U^i \rangle$ and cross moments $\langle x U^i \rangle$. Note that the second dimension, "umom", short for "U moments", is just `order` plus 1. That makes sense if we remember that the zeroth order derivative is the observable itself and we specify that we want to use up to `order` derivatives. So the second dimension involves setting the exponent $i$ on $U$ equal to the index of that dimension, or order of that moment. The first dimension does the same thing, but with $x$. Irregardless of the order, however, we only ever need $x$ raised to the zeroth or first power in the average.

To summarize, the first row in `.values` contains all moments of just $U$, i.e., $\langle U^0 \rangle$, $\langle U^1 \rangle$, $\langle U^2 \rangle$, etc. The second row in `.values` contains all moments of $x$ multiplied by $U$, i.e., $\langle x U^0 \rangle$, $\langle x U^1 \rangle$, etc.  But note that beyond the powers of 0 and 1 for the first row, and just 0 for the second row, all values shown are central moments, e.g., $\langle (x - \langle x \rangle)(U - \langle U \rangle)^i \rangle$ or $\langle (U - \langle U \rangle)^i \rangle$.

<font color="red">Is the average of U to the zeroth power a bug?  Doesn't seem to mess anything up because maybe never gets used... but still concerning because should just be 1, but looks like didn't get divided by the number of samples...</font>

Since this is the internal structure, you can just provide it directly (or something similar to it in terms of moments) if you prefer to do that. You'll just need to use a different constuctor method than `from_vals`.

A big caveat, though. All uncertainty estimation happens through bootstrap resampling along the 'rec' dimension of the originally provided data. So if you just pass in the central moments above, you won't be able to calculate uncertainties...

In [95]:
data_noboot = xpan_beta.DataCentralMoments.from_raw(data.values)

data_noboot.resample(nrep=3).values

ValueError: 'rec' not found in array dimensions ('xmom', 'umom')

versus...

In [102]:
data.resample(nrep=3).values

<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74900468e+02, 2.79822661e+01],
        [1.74900468e-01, 2.79822661e-02, 7.06016530e-03]],

       [[1.00000000e+05, 1.74874054e+02, 2.75343045e+01],
        [1.74874054e-01, 2.75343045e-02, 7.14831079e-03]],

       [[1.00000000e+05, 1.74861517e+02, 2.78405751e+01],
        [1.74861517e-01, 2.78405751e-02, 7.28200877e-03]]])
Dimensions without coordinates: rep, xmom, umom

Even if you add a 'rec' dimension, bootstrapping with a single observation ('rec' dim of size 1) won't do anything...

In [103]:
data_noboot = xpan_beta.DataCentralMoments.from_raw(xr.DataArray(data.values.values[None, ...], dims=['rec', 'xmom', 'umom']))

In [108]:
data_noboot.resample(nrep=3).values

<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[ 1.00000000e+05,  1.74863233e+02, -3.05491896e+04],
        [ 1.74863233e-01, -3.05491896e+01,  1.06789780e+04]],

       [[ 1.00000000e+05,  1.74863233e+02, -3.05491896e+04],
        [ 1.74863233e-01, -3.05491896e+01,  1.06789780e+04]],

       [[ 1.00000000e+05,  1.74863233e+02, -3.05491896e+04],
        [ 1.74863233e-01, -3.05491896e+01,  1.06789780e+04]]])
Dimensions without coordinates: rep, xmom, umom

Resampling in this case just gives the same thing every time!

Note that whenever you call `.resample()` a new dimension is created for the output called 'rep', short for "repetitions." This is similar to the 'rec' dimension, but helps you keep track of whether you're working with original data, or a boostrapped sample.

So clearly if you like to provide raw moments, you will need to do this from blocks of your data or from repeated simulations if you want the uncertainty quantification in `thermoextrap` to work.

But if you do like to calculate your own moments (or save them periodically from simulations rather than saving all of the configurations, energies, or observable frequently), you can also make that work.

But first, we just work with blocks of data, adding an axis called 'block' that we will ask the constructor to average over by specifying the `axis` argument.

In [112]:
# Make 100 averaged observations
xx = xr.DataArray(xdata.values.reshape(100, -1), dims=['rec', 'block'])
uu = xr.DataArray(udata.values.reshape(100, -1), dims=['rec', 'block'])

In [114]:
# Create directly from values of moments - notice that this is DataCentralMoments, not DataCentralMomentsVals
# Effectively just means that the 'rec' dim will not be collapsed when using data for extrapolation, etc.
# Behaves similarly to the 'rep' dim when resampling
data_fv = xpan_beta.DataCentralMoments.from_vals(xv=xx, uv=uu, axis='block', order=order, central=True)
# So 'rec' is for each separate block average
data_fv.values

<xarray.DataArray (rec: 100, xmom: 2, umom: 3)>
array([[[ 1.00000000e+03,  1.74927602e+02,  2.97138187e+01],
        [ 1.74927602e-01,  2.97138187e-02, -2.00545408e-04]],

       [[ 1.00000000e+03,  1.75019131e+02,  2.67043890e+01],
        [ 1.75019131e-01,  2.67043890e-02,  1.28682639e-02]],

       [[ 1.00000000e+03,  1.75051102e+02,  2.81614547e+01],
        [ 1.75051102e-01,  2.81614547e-02, -1.35243356e-02]],

       [[ 1.00000000e+03,  1.74811567e+02,  3.02060251e+01],
        [ 1.74811567e-01,  3.02060251e-02, -2.47450817e-03]],

       [[ 1.00000000e+03,  1.74809254e+02,  2.68642935e+01],
        [ 1.74809254e-01,  2.68642935e-02,  1.84412990e-02]],

       [[ 1.00000000e+03,  1.74913534e+02,  2.82670008e+01],
        [ 1.74913534e-01,  2.82670008e-02, -9.80703638e-05]],

       [[ 1.00000000e+03,  1.75009921e+02,  2.68787369e+01],
        [ 1.75009921e-01,  2.68787369e-02,  1.55895309e-02]],
...
       [[ 1.00000000e+03,  1.74787839e+02,  2.80889472e+01],
        [ 1.74787839e-01,  2.80889472e-02,  2.86422238e-02]],

       [[ 1.00000000e+03,  1.74683396e+02,  2.91992311e+01],
        [ 1.74683396e-01,  2.91992311e-02,  1.35462738e-02]],

       [[ 1.00000000e+03,  1.74945634e+02,  2.70342312e+01],
        [ 1.74945634e-01,  2.70342312e-02,  2.00515096e-03]],

       [[ 1.00000000e+03,  1.74738398e+02,  2.83023014e+01],
        [ 1.74738398e-01,  2.83023014e-02,  4.04833298e-03]],

       [[ 1.00000000e+03,  1.75083307e+02,  2.84374878e+01],
        [ 1.75083307e-01,  2.84374878e-02, -4.46414800e-03]],

       [[ 1.00000000e+03,  1.74780432e+02,  2.79356195e+01],
        [ 1.74780432e-01,  2.79356195e-02,  1.56856074e-02]],

       [[ 1.00000000e+03,  1.74781495e+02,  2.90386427e+01],
        [ 1.74781495e-01,  2.90386427e-02, -1.57766179e-02]]])
Dimensions without coordinates: rec, xmom, umom

What's different here is that the 'rec' dim also appears in `.values`. That means that when this data is used in models for extrapolation or interpolation, that dimension will also be preserved. So prediction to a new $\beta$ value will result in an output matching the size of the 'rec' dimension in the same way that it would match the 'rep' dimension created through resampling.

If we resample over this data set, we see that we just take `nrep` random samples from it, putting those samples into a new dimension called 'rep'.

In [115]:
data_fv.resample(nrep=3).values

<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74857524e+02, 2.81821991e+01],
        [1.74857524e-01, 2.81821991e-02, 6.72454182e-03]],

       [[1.00000000e+05, 1.74856541e+02, 2.78909295e+01],
        [1.74856541e-01, 2.78909295e-02, 5.93380461e-03]],

       [[1.00000000e+05, 1.74860765e+02, 2.78098181e+01],
        [1.74860765e-01, 2.78098181e-02, 6.75192301e-03]]])
Dimensions without coordinates: rep, xmom, umom

Now if we had computed the moments from the blocked data ourselves, we can also create a data object with the `from_ave_raw` constructor (below). Many other constructors exist, including from central moments if you like. If you use those, please take a look at the documentation to make sure you are specifying or using the correct dimension naming conventions, such as 'rec', 'xmom', 'umom', etc. Remember, if you are extrapolating an observable that has an explicit dependence on the extrapolation observable, you also need to specify the 'deriv' dimension that describes the observed derivatives with respect to the extrapolation variable (see the "Temperature_Extrap_Case2" notebook).

In [116]:
# Compute moments of U, i.e., averages to integer powers up to maximum order desired
mom_u = xr.DataArray(np.arange(order + 1), dims=['umom'])
uave = (uu ** mom_u).mean('block')
xuave = (xx * uu ** mom_u).mean('block')
data_fa = xpan_beta.DataCentralMoments.from_ave_raw(u=uave, xu=xuave, central=True)

In [117]:
data_fa.values

<xarray.DataArray (rec: 100, xmom: 2, umom: 3)>
array([[[ 1.00000000e+00,  1.74927602e+02,  2.97138187e+01],
        [ 1.74927602e-01,  2.97138187e-02, -2.00545398e-04]],

       [[ 1.00000000e+00,  1.75019131e+02,  2.67043890e+01],
        [ 1.75019131e-01,  2.67043890e-02,  1.28682638e-02]],

       [[ 1.00000000e+00,  1.75051102e+02,  2.81614547e+01],
        [ 1.75051102e-01,  2.81614547e-02, -1.35243356e-02]],

       [[ 1.00000000e+00,  1.74811567e+02,  3.02060251e+01],
        [ 1.74811567e-01,  3.02060251e-02, -2.47450818e-03]],

       [[ 1.00000000e+00,  1.74809254e+02,  2.68642935e+01],
        [ 1.74809254e-01,  2.68642935e-02,  1.84412990e-02]],

       [[ 1.00000000e+00,  1.74913534e+02,  2.82670008e+01],
        [ 1.74913534e-01,  2.82670008e-02, -9.80704108e-05]],

       [[ 1.00000000e+00,  1.75009921e+02,  2.68787369e+01],
        [ 1.75009921e-01,  2.68787369e-02,  1.55895309e-02]],
...
       [[ 1.00000000e+00,  1.74787839e+02,  2.80889472e+01],
        [ 1.74787839e-01,  2.80889472e-02,  2.86422238e-02]],

       [[ 1.00000000e+00,  1.74683396e+02,  2.91992311e+01],
        [ 1.74683396e-01,  2.91992311e-02,  1.35462738e-02]],

       [[ 1.00000000e+00,  1.74945634e+02,  2.70342312e+01],
        [ 1.74945634e-01,  2.70342312e-02,  2.00515097e-03]],

       [[ 1.00000000e+00,  1.74738398e+02,  2.83023014e+01],
        [ 1.74738398e-01,  2.83023014e-02,  4.04833296e-03]],

       [[ 1.00000000e+00,  1.75083307e+02,  2.84374878e+01],
        [ 1.75083307e-01,  2.84374878e-02, -4.46414801e-03]],

       [[ 1.00000000e+00,  1.74780432e+02,  2.79356195e+01],
        [ 1.74780432e-01,  2.79356195e-02,  1.56856074e-02]],

       [[ 1.00000000e+00,  1.74781495e+02,  2.90386427e+01],
        [ 1.74781495e-01,  2.90386427e-02, -1.57766179e-02]]])
Dimensions without coordinates: rec, xmom, umom

The above `.values` should be identical to those from the `from_vals` constructor.

At this point, we have seen how the same data objects that interface with extrapolation or interpolation models can be created from different inputs. Other features also exist, such as specifying weights with the argument `w` to a constructor to change the weights used during averaging.

Finally, we can also have vector-based observables, like RDFs, for example. This is easy to accomplish with any of the above constructors or types of data input. All that is required is to add another dimension to our xarray `DataArray` input. Typically, we will call this dimension 'vals', short for "values", which is the default name for this dimension when using the `DataCentralMomentVals.from_vals` constructor.

In [127]:
# Extrapolate both average x and average x**2
x_xsq_data = xr.DataArray(np.vstack([xdata.values, xdata.values**2]).T, dims=['rec', 'vals'], coords={'vals': ['x','xsq']})
data_vec = xpan_beta.DataCentralMomentsVals.from_vals(order=order, 
                                                  rec_dim='rec',
                                                  xv=x_xsq_data, uv=udata, central=True)

In [128]:
data_vec.values

<xarray.DataArray (vals: 2, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74863233e+02, 2.79605493e+01],
        [1.74863233e-01, 2.79605493e-02, 7.18138659e-03]],

       [[1.00000000e+05, 1.74863233e+02, 2.79605493e+01],
        [3.06051107e-02, 9.78572548e-03, 4.07733335e-03]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: xmom, umom

In [129]:
data_vec.resample(nrep=3).values

<xarray.DataArray (rep: 3, vals: 2, xmom: 2, umom: 3)>
array([[[[1.00000000e+05, 1.74872168e+02, 2.78881919e+01],
         [1.74872168e-01, 2.78881919e-02, 5.76861181e-03]],

        [[1.00000000e+05, 1.74872168e+02, 2.78881919e+01],
         [3.06081632e-02, 9.75950575e-03, 3.57590083e-03]]],


       [[[1.00000000e+05, 1.74833251e+02, 2.78315782e+01],
         [1.74833251e-01, 2.78315782e-02, 6.62533125e-03]],

        [[1.00000000e+05, 1.74833251e+02, 2.78315782e+01],
         [3.05944973e-02, 9.73839594e-03, 3.86544019e-03]]],


       [[[1.00000000e+05, 1.74874332e+02, 2.79524497e+01],
         [1.74874332e-01, 2.79524497e-02, 7.96927064e-03]],

        [[1.00000000e+05, 1.74874332e+02, 2.79524497e+01],
         [3.06089844e-02, 9.78430122e-03, 4.36295394e-03]]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: rep, xmom, umom

Note that we have simply added a dimension along which all the same operations happen, but independently for different data. The behavior is identical if we work instead with data from other constructors.

In [143]:
xx_xsqxsq = xr.DataArray(x_xsq_data.values.reshape(100, -1, 2), dims=['rec', 'block', 'vals'], coords={'vals': ['x','xsq']})
x_xsq_uave = (xx_xsqxsq * uu ** mom_u).mean('block')
data_fa_vec = xpan_beta.DataCentralMoments.from_ave_raw(u=uave, xu=x_xsq_uave, central=True)

In [149]:
data_fa_vec.resample(nrep=3).values

<xarray.DataArray (rep: 3, vals: 2, xmom: 2, umom: 3)>
array([[[[1.00000000e+02, 1.74868314e+02, 2.80069285e+01],
         [1.74868314e-01, 2.80069285e-02, 6.78162216e-03]],

        [[1.00000000e+02, 1.74868314e+02, 2.80069285e+01],
         [3.06069341e-02, 9.80183033e-03, 3.93242766e-03]]],


       [[[1.00000000e+02, 1.74885615e+02, 2.79407211e+01],
         [1.74885615e-01, 2.79407211e-02, 7.14753138e-03]],

        [[1.00000000e+02, 1.74885615e+02, 2.79407211e+01],
         [3.06129190e-02, 9.78000791e-03, 4.08292796e-03]]],


       [[[1.00000000e+02, 1.74872887e+02, 2.80647473e+01],
         [1.74872887e-01, 2.80647473e-02, 5.39603998e-03]],

        [[1.00000000e+02, 1.74872887e+02, 2.80647473e+01],
         [3.06085914e-02, 9.82092281e-03, 3.45404670e-03]]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: rep, xmom, umom

<font color="red">What just happened above?</font>

Not sure if I'm missing anything, but this should be what you we're after.
Might have been a bug somewhere previously.